<a href="https://colab.research.google.com/github/Omarrhussain/SYNC-S-Internship/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
with open('/content/Intent.json') as f:
    intents = json.load(f)

In [ ]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [ ]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []

    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])

    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [ ]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

    tokenizer.fit_on_texts(input_list)

    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')

    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [ ]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])

    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [ ]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


In [ ]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [ ]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirection  (None, 256)               656384    
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758166 (2.89 MB)
Trainable params: 758166 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 5s 49ms/step - loss: 3.0861 - accuracy: 0.0629
Epoch 2/50
5/5 [==============================] - 0s 54ms/step - loss: 3.0612 - accuracy: 0.1329
Epoch 3/50
5/5 [==============================] - 0s 52ms/step - loss: 3.0194 - accuracy: 0.1678
Epoch 4/50
5/5 [==============================] - 0s 53ms/step - loss: 2.9823 - accuracy: 0.1538
Epoch 5/50
5/5 [==============================] - 0s 52ms/step - loss: 2.8890 - accuracy: 0.3357
Epoch 6/50
5/5 [==============================] - 0s 63ms/step - loss: 2.8100 - accuracy: 0.2727
Epoch 7/50
5/5 [==============================] - 0s 86ms/step - loss: 2.6120 - accuracy: 0.4476
Epoch 8/50
5/5 [==============================] - 0s 81ms/step - loss: 2.3723 - accuracy: 0.3706
Epoch 9/50
5/5 [==============================] - 0s 80ms/step - loss: 2.2048 - accuracy: 0.4336
Epoch 10/50
5/5 [==============================] - 0s 78ms/step - loss: 1.8712 - accuracy: 0.4895
Epoch 11/50
5/5 [============

In [ ]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))

    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)

    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.
You: Hello
Bot: Let me see -- TYPE: WhoAmI

You: how are you ?
Bot: Hello, how are you? I am great thanks! Please tell me your GeniSys user -- TYPE: CourtesyGreeting

You:  what's your name ?
Bot: GeniSys -- TYPE: RealNameQuery

You: no i want your real name?
Bot: My name is GeniSys -- TYPE: RealNameQuery

You:  Can you tell me if you are self-aware ?
Bot: That is an interesting question, can you prove that you are? -- TYPE: SelfAware

You: tell me a joke
Bot: A doctor thoroughly examined his patient and said, 'Look I really can't find any reason for this mysterious affliction. It's probably due to drinking.' The patient sighed and snapped, 'In that case, I'll come back when you're damn well sober!' -- TYPE: Jokes

You: can you open the door please ?
Bot: I’m sorry, I’m afraid I can’t do that! -- TYPE: PodBayDoor

You: quit
